In [3]:
from nltk.tokenize import word_tokenize
import nltk

In [4]:
from konlpy.tag import Okt

In [5]:
pos_tagger = Okt()

In [6]:
train = [('메리가 좋아', 'pos'), 
         ('고양이도 좋아', 'pos'),
         ('난 수업이 지루해', 'neg'),
         ('메리는 이쁜 고양이야', 'pos'),
         ('난 마치고 메리랑 놀거야', 'pos')]

위에 있는 train을 가지고 말뭉치를 만들어봅니다

In [7]:
# train문장에서 사용된 전체 단어를 찾습니다.
all_words = set(word.lower() for sentence in train
                        for word in word_tokenize(sentence[0]))
all_words

{'고양이도',
 '고양이야',
 '난',
 '놀거야',
 '마치고',
 '메리가',
 '메리는',
 '메리랑',
 '수업이',
 '이쁜',
 '좋아',
 '지루해'}

In [8]:
# 말뭉치를 기준으로 train문장에 속한 단어인지 아닌지를 기록합니다.
t = [({word: (word in word_tokenize(x[0])) for word in all_words}, x[1])
                                                        for x in train]
t

[({'고양이도': False,
   '놀거야': False,
   '고양이야': False,
   '난': False,
   '이쁜': False,
   '지루해': False,
   '수업이': False,
   '메리는': False,
   '좋아': True,
   '메리가': True,
   '메리랑': False,
   '마치고': False},
  'pos'),
 ({'고양이도': True,
   '놀거야': False,
   '고양이야': False,
   '난': False,
   '이쁜': False,
   '지루해': False,
   '수업이': False,
   '메리는': False,
   '좋아': True,
   '메리가': False,
   '메리랑': False,
   '마치고': False},
  'pos'),
 ({'고양이도': False,
   '놀거야': False,
   '고양이야': False,
   '난': True,
   '이쁜': False,
   '지루해': True,
   '수업이': True,
   '메리는': False,
   '좋아': False,
   '메리가': False,
   '메리랑': False,
   '마치고': False},
  'neg'),
 ({'고양이도': False,
   '놀거야': False,
   '고양이야': True,
   '난': False,
   '이쁜': True,
   '지루해': False,
   '수업이': False,
   '메리는': True,
   '좋아': False,
   '메리가': False,
   '메리랑': False,
   '마치고': False},
  'pos'),
 ({'고양이도': False,
   '놀거야': True,
   '고양이야': False,
   '난': True,
   '이쁜': False,
   '지루해': False,
   '수업이': False,
   '메리는': False,
   '좋아': False,
   '메리가':

In [9]:
classifier = nltk.NaiveBayesClassifier.train(t)
classifier.show_most_informative_features()

Most Informative Features
                       난 = True              neg : pos    =      2.5 : 1.0
                      좋아 = False             neg : pos    =      1.5 : 1.0
                      이쁜 = False             neg : pos    =      1.1 : 1.0
                     메리는 = False             neg : pos    =      1.1 : 1.0
                    고양이야 = False             neg : pos    =      1.1 : 1.0
                    고양이도 = False             neg : pos    =      1.1 : 1.0
                     마치고 = False             neg : pos    =      1.1 : 1.0
                     놀거야 = False             neg : pos    =      1.1 : 1.0
                     메리랑 = False             neg : pos    =      1.1 : 1.0
                     메리가 = False             neg : pos    =      1.1 : 1.0


In [10]:
test_sentence = '난 수업이 마치면 메리랑 놀거야'

In [11]:
test_sent_features = {word.lower(): 
                                          (word in word_tokenize(test_sentence.lower()))
                                           for word in all_words}
test_sent_features

{'고양이도': False,
 '놀거야': True,
 '고양이야': False,
 '난': True,
 '이쁜': False,
 '지루해': False,
 '수업이': True,
 '메리는': False,
 '좋아': False,
 '메리가': False,
 '메리랑': True,
 '마치고': False}

In [12]:
classifier.classify(test_sent_features)

'neg'

위의 결과를 종합해보면 한글을 다룰 때는 형태소 분석이 필요하다는 결론을 얻을 수 있습니다

In [16]:
# tokenize함수를 만들어줍니다
# 태그를 붙여주는 것이 유리합니다
def tokenize(doc):
    return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]

In [17]:
# tokenize함수를 이용해서 train문장을 분석합니다
train_docs = [(tokenize(row[0]), row[1]) for row in train]
train_docs

[(['메리/Noun', '가/Josa', '좋다/Adjective'], 'pos'),
 (['고양이/Noun', '도/Josa', '좋다/Adjective'], 'pos'),
 (['난/Noun', '수업/Noun', '이/Josa', '지루하다/Adjective'], 'neg'),
 (['메리/Noun', '는/Josa', '이쁘다/Adjective', '고양이/Noun', '야/Josa'], 'pos'),
 (['난/Noun', '마치/Noun', '고/Josa', '메리/Noun', '랑/Josa', '놀다/Verb'], 'pos')]

In [18]:
# 전체 말뭉치도 만들어줍니다
tokens = [t for d in train_docs for t in d[0]]
tokens

['메리/Noun',
 '가/Josa',
 '좋다/Adjective',
 '고양이/Noun',
 '도/Josa',
 '좋다/Adjective',
 '난/Noun',
 '수업/Noun',
 '이/Josa',
 '지루하다/Adjective',
 '메리/Noun',
 '는/Josa',
 '이쁘다/Adjective',
 '고양이/Noun',
 '야/Josa',
 '난/Noun',
 '마치/Noun',
 '고/Josa',
 '메리/Noun',
 '랑/Josa',
 '놀다/Verb']

In [19]:
# 말뭉치에 있는 단어가 있는지 아난지를 구분하는 함수를 만들어줍니다
def term_exists(doc):
    return {word: (word in set(doc)) for word in tokens}

In [20]:
# term_exist함수를 이용해서 train문장에 적용해 봅니다
# 조사(josa)인지 명사(noun)인지 구분이 잘 되어 있어서 판독하기 좋습니다
train_xy = [(term_exists(d), c) for d,c in train_docs]
train_xy

[({'메리/Noun': True,
   '가/Josa': True,
   '좋다/Adjective': True,
   '고양이/Noun': False,
   '도/Josa': False,
   '난/Noun': False,
   '수업/Noun': False,
   '이/Josa': False,
   '지루하다/Adjective': False,
   '는/Josa': False,
   '이쁘다/Adjective': False,
   '야/Josa': False,
   '마치/Noun': False,
   '고/Josa': False,
   '랑/Josa': False,
   '놀다/Verb': False},
  'pos'),
 ({'메리/Noun': False,
   '가/Josa': False,
   '좋다/Adjective': True,
   '고양이/Noun': True,
   '도/Josa': True,
   '난/Noun': False,
   '수업/Noun': False,
   '이/Josa': False,
   '지루하다/Adjective': False,
   '는/Josa': False,
   '이쁘다/Adjective': False,
   '야/Josa': False,
   '마치/Noun': False,
   '고/Josa': False,
   '랑/Josa': False,
   '놀다/Verb': False},
  'pos'),
 ({'메리/Noun': False,
   '가/Josa': False,
   '좋다/Adjective': False,
   '고양이/Noun': False,
   '도/Josa': False,
   '난/Noun': True,
   '수업/Noun': True,
   '이/Josa': True,
   '지루하다/Adjective': True,
   '는/Josa': False,
   '이쁘다/Adjective': False,
   '야/Josa': False,
   '마치/Noun': False,
   '고/Jo

In [21]:
# 분류기를 작동시켜봅니다
classifier = nltk.NaiveBayesClassifier.train(train_xy)

In [23]:
# 형태소 분석을 안했을 때에 부정되었던 문장을 다시 가지고 테스트해봅니다
test_sentence = [("난 수업이 마치면 메리랑 놀거야")]

In [30]:
# 형태소 분석을 해봅니다
test_docs = pos_tagger.pos(test_sentence[0])
test_docs

[('난', 'Noun'),
 ('수업', 'Noun'),
 ('이', 'Josa'),
 ('마치', 'Noun'),
 ('면', 'Josa'),
 ('메리', 'Noun'),
 ('랑', 'Josa'),
 ('놀거야', 'Verb')]

In [31]:
classifier.show_most_informative_features()

Most Informative Features
                 메리/Noun = False             neg : pos    =      2.5 : 1.0
                  난/Noun = True              neg : pos    =      2.5 : 1.0
            좋다/Adjective = False             neg : pos    =      1.5 : 1.0
                고양이/Noun = False             neg : pos    =      1.5 : 1.0
                 놀다/Verb = False             neg : pos    =      1.1 : 1.0
                  고/Josa = False             neg : pos    =      1.1 : 1.0
                  는/Josa = False             neg : pos    =      1.1 : 1.0
                  랑/Josa = False             neg : pos    =      1.1 : 1.0
                  야/Josa = False             neg : pos    =      1.1 : 1.0
                 마치/Noun = False             neg : pos    =      1.1 : 1.0


In [32]:
test_sent_features = {word: (word in tokens) for word in test_docs}
test_sent_features

{('난', 'Noun'): False,
 ('수업', 'Noun'): False,
 ('이', 'Josa'): False,
 ('마치', 'Noun'): False,
 ('면', 'Josa'): False,
 ('메리', 'Noun'): False,
 ('랑', 'Josa'): False,
 ('놀거야', 'Verb'): False}

In [33]:
classifier.classify(test_sent_features)

'pos'